<a href="https://colab.research.google.com/github/Nazhan99/Tensorflow_notes/blob/main/08_Intro_to_NLP_in_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (could be sdequences text or speech)

Another common term for NLP problems is sequence to sequence problems (seq2seq)

##Check for GPU


In [1]:
!nvidia-smi

Sun Mar  6 06:48:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

#import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves , compare_historys

--2022-03-06 06:48:02--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-03-06 06:48:02 (114 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The data set we are going to be using is Kaggle's introduction to NLP dataset (text samples of tweets labelled as disaster or not disaster)

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#unzip data
unzip_data("nlp_getting_started.zip")

--2022-03-06 06:48:05--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.135.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.135.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-03-06 06:48:05 (137 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualize a text dataset

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [5]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [7]:
#shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
#how many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
#how many total samples?
len(train_df)

7613

In [11]:
len(test_df)

3263

In [12]:
#let's visualize some random training examples
import random 
random_index= random.randint(0,len(train_df)-5) #create random indexes not higeher than total samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n {text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
 Experts in France begin examining airplane debris found on Reunion Island: French air accident experts on Wedn... http://t.co/bKpFpOGySI

---

Target: 0 (not real disaster)
Text:
 For those that were interested in the gun powder art discussed at the end of 'Introduction to Theological Aesthetic' http://t.co/BZ3iR4GMWj

---

Target: 0 (not real disaster)
Text:
 @RaynbowAffair Editor In Chief @DiamondKesawn Releases Issue #7 http://t.co/7mzYcU2IHo of #RAmag. #Fashion #Models and #Mayhem

---

Target: 0 (not real disaster)
Text:
 Do you have an emergency drinking water plan? Download guide in English Spanish French Arabic or Vietnamese. http://t.co/S0ktilisKq

---

Target: 0 (not real disaster)
Text:
 Seen on Fahlo:#WCW All Hail the QueenåÊ?? http://t.co/oLpBmy9xw9 #MTVHottest Justin Bieber http://t.co/ON18cqGcoA

---



### Split data into training and validation sets

In [13]:
from sklearn.model_selection import train_test_split


In [14]:
#use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                            train_df_shuffled["target"].to_numpy(),
                                            test_size=0.1, #use 10% of training data for validation
                                            random_state=42)

In [15]:
#check the lengths
len(train_sentences) , len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [16]:
#check the first 10 samples 
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

When dealing with  a text problem, one of the first things you will have to do before you can build a model is to conver tthe text to numbers.

There are a few way to do:

* Tokenization - direct mapping of token( a token could be a word or a character) to number
* Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

## Text vectorization (tokenization)

In [17]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [18]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

#https://tensorflow.google.cn/api_docs/python/tf/keras/layers/TextVectorization

In [19]:
#Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=50,# how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, #create groups of n-words?
                                    output_mode="int" ,#how to map tokens to numbers
                                    output_sequence_length=None, #how long do you want your sequences to be
                                    pad_to_max_tokens=True)

In [20]:
#find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [21]:
#setup text vectorization variables
max_vocab_length = 10000 #max number of words to have in our vocabualry
max_length= 15 #max length our sequences will be ( how many words from a tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                     output_mode="int",
                                     output_sequence_length=max_length)

In [22]:
#fit the text vectorizer 
text_vectorizer.adapt(train_sentences)

In [23]:
#create a sample sentence and tokenize it 
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [24]:
#Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 I rate Hazard very highly but his fanboys are among the worst accounts on Twitter.        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   8, 1264,  423,  591, 3779,   30,   79,    1,   22, 1558,    2,
         971, 6275,   11,  382]])>

In [25]:
#get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all the unique word 
top_5_words = words_in_vocab[:5] #get the most common words
bottom_5_words = words_in_vocab[:-5] # get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {len(top_5_words)}, {top_5_words}")
print(f"5 least common words: {len(bottom_5_words)}, {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: 5, ['', '[UNK]', 'the', 'a', 'in']
5 least common words: 9995, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is', 'for', 'on', 'you', 'my', 'with', 'it', 'that', 'at', 'by', 'this', 'from', 'be', 'are', 'was', 'have', 'like', 'as', 'up', 'so', 'just', 'but', 'me', 'im', 'your', 'not', 'amp', 'out', 'its', 'will', 'an', 'no', 'has', 'fire', 'after', 'all', 'when', 'we', 'if', 'now', 'via', 'new', 'more', 'get', 'or', 'about', 'what', 'he', 'people', 'news', 'been', 'over', 'one', 'how', 'dont', 'they', 'who', 'into', 'were', 'do', 'us', '2', 'can', 'video', 'emergency', 'there', 'disaster', 'than', 'police', 'would', 'his', 'still', 'her', 'some', 'body', 'storm', 'crash', 'burning', 'suicide', 'back', 'man', 'california', 'why', 'time', 'them', 'had', 'buildings', 'rt', 'first', 'cant', 'see', 'got', 'day', 'off', 'our', 'going', 'nuclear', 'know', 'world', 'bomb', 'fires', 'love', 'killed', 'go', 'attack', 'youtube', 'dea

## Creating an embedding using a Embedding Layer

To make our embedding, we are going to use tensorflow embedding layer

refer: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would be 100 long,
* `input_length` = length of the sequences being passed to the embedding layer

In [26]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, #set input shape
                             embeddings_initializer= "uniform",
                             output_dim=128,
                             input_length=max_length #how long is each input
                             )

embedding

In [27]:
#get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      \n\nEmbedded version:")

#embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Teen Disaster Preparedness Event in Van Nuys August 11 @ 5:30pm http://t.co/fXUX987vZx via @VanNuysCouncil      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04502403,  0.02926755, -0.01321647, ...,  0.04638353,
          0.0226543 , -0.0207734 ],
        [ 0.04969288, -0.02301311,  0.00780017, ..., -0.0325652 ,
         -0.00188253, -0.03146359],
        [-0.04886279,  0.02465316, -0.04275448, ..., -0.04132904,
         -0.04654399, -0.04649109],
        ...,
        [-0.03257634, -0.01262859,  0.03865603, ..., -0.03575226,
          0.01477912, -0.04538474],
        [-0.01171023, -0.04764355, -0.00410885, ...,  0.00144283,
          0.02976141, -0.00853884],
        [-0.01171023, -0.04764355, -0.00410885, ...,  0.00144283,
          0.02976141, -0.00853884]]], dtype=float32)>

In [28]:
#check out a singel token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.04502403,  0.02926755, -0.01321647,  0.00137496, -0.04722915,
         0.02575585, -0.00350557,  0.00466088, -0.02238203,  0.01753167,
         0.03738787,  0.0221366 , -0.04703884,  0.01013148,  0.04081242,
        -0.02815312,  0.02465175,  0.03127478, -0.03432596, -0.02220701,
         0.0206578 , -0.03343161,  0.00467614, -0.01598163, -0.02524153,
         0.03403321,  0.03362293,  0.01500798,  0.03286738, -0.03100877,
        -0.0368363 ,  0.04625484, -0.0327444 , -0.01047192,  0.00749386,
        -0.03287782,  0.0049948 , -0.02527833,  0.0195994 , -0.00153454,
         0.01181079, -0.0244355 , -0.04200619,  0.02455064, -0.04886434,
        -0.04518007,  0.0496583 , -0.02996118,  0.0338175 ,  0.03808169,
         0.02594015,  0.0123395 , -0.01934849, -0.03760674,  0.04603766,
         0.04176512,  0.01564045, -0.04108166, -0.04366759, -0.03580499,
         0.01401886,  0.03167841,  0.03650634, -0.02696147, -0.01370685,
  

## Modelling a text dataset (running a series of experiments)

Now we have got way to turn our text sequences into numbers, it is time to start building a series of modelling experiments.

We will start with a baseline and move on from there.

* Model 0: Naive Bayes(baseline, this is from sklearn ML map: 
* Model 1: Feed=forward Neural Network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: Tensorflow Hub pretrained feature extractor (using transfer learning for NLP)
* Model 7: same as model 6 with 10% of training data

how are we going to approach all of these?

Use the standard steps in modelling with tensorflow:
* Create a model
* Build a model
* Fit a model
* Evaluate our model

## Model 0: Getting a baseline

As with all machine learning modelling experiments, it is important to create a baseline model so you have got a benchmark for future experiments to build upon.

To create our baseline, we will use Sklearn's Multinormial Naive Bayes using the T-IDF ormula to convert our words to numbers.

>Note: It's common practice to use non-DL algorithms as a baseline because of their speed and then later using DL to see if you can improve upon them.

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), #convert words to numebers using tfidf
                    ("clf", MultinomialNB()) #model the text, "clf" stands for classifier

])

#Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels )

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [30]:
#Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [31]:
#make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation functions for our model experiments

We could evaluate all of our model's predictions with different metrics every time,  however this will be cumbersome and could easily fix with a function.

Let's create one to compare our model's predicitons with the truth labels using the following metrics:
* Accuracy
* Precision
* Recall
* F1-score

For a deep overview of many different evalaution methods, see the Sklearn documentation: https://scikit-learn.org/stable/model_selection.html

In [32]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculate model accuracy, precision, recall and 1-core of a binary classification model
  """

  #calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred)*100

  #calculate model precision, recall and 1-score using "weighted" average
  model_precision, model_recall, model_f1, _ =precision_recall_fscore_support(y_true,y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results


In [33]:
#get baseline results
baseline_results = calculate_results(y_true= val_labels, 
                                     y_pred= baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

## Model 1: A simple dense model



In [34]:
#create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

#create a directory to save TensorBoard logs
SAVE_DIR ="model_logs"

In [35]:
#build model with the Funtional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) #inputs are 1-dimensional strings
x =text_vectorizer(inputs) #turn the input text into numbers
x =embedding(x) #create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D(name="global_avg_pool_layer")(x)
outputs = layers.Dense(1, activation="sigmoid")(x) #create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [36]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_avg_pool_layer (Glob  (None, 128)              0         
 alAveragePooling1D)                                             
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [37]:
#compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
  

In [38]:
#fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20220306-064809
Epoch 1/5
215/215 [==============================] - 6s 9ms/step - loss: 0.6109 - accuracy: 0.6927 - val_loss: 0.5380 - val_accuracy: 0.7441
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.4414 - accuracy: 0.8155 - val_loss: 0.4693 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3469 - accuracy: 0.8587 - val_loss: 0.4573 - val_accuracy: 0.7927
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2851 - accuracy: 0.8894 - val_loss: 0.4647 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2381 - accuracy: 0.9114 - val_loss: 0.4797 - val_accuracy: 0.7835


In [39]:
#check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 6ms/step - loss: 0.4797 - accuracy: 0.7835


[0.4797159731388092, 0.7834645509719849]

In [40]:
#make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

(762, 1)

In [41]:
model_1_pred_probs[:10]

array([[0.38535208],
       [0.78375566],
       [0.99786335],
       [0.1315021 ],
       [0.11714505],
       [0.9500229 ],
       [0.9035294 ],
       [0.992467  ],
       [0.96351355],
       [0.30665883]], dtype=float32)

In [42]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_avg_pool_layer (Glob  (None, 128)              0         
 alAveragePooling1D)                                             
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [43]:
#convert  model predictions probs to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [44]:
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [45]:
#calculate our model_1_results
model_1_results = calculate_results(y_true = val_labels,
                                    y_pred=model_1_preds)

In [46]:
model_1_results

{'accuracy': 78.34645669291339,
 'f1': 0.7807800582578167,
 'precision': 0.7872123378365872,
 'recall': 0.7834645669291339}

In [47]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [48]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## Visualizing learned embeddings

In [49]:
# get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [50]:
#model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_avg_pool_layer (Glob  (None, 128)              0         
 alAveragePooling1D)                                             
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [51]:
# get the weight matrix of embeding layer
#get the numerical representations of each token in our trianing data
embed_weights= model_1.get_layer("embedding").get_weights()[0]
embed_weights

array([[ 0.00038442, -0.03450402, -0.0173611 , ...,  0.01210326,
         0.03702473,  0.00333397],
       [ 0.01544648,  0.0056816 ,  0.0079445 , ..., -0.01954085,
         0.05991372, -0.0016805 ],
       [ 0.02424764,  0.00382106,  0.02181069, ...,  0.05986955,
         0.01941948, -0.02600526],
       ...,
       [ 0.03555247, -0.04774951,  0.01095488, ..., -0.00063627,
         0.00300682, -0.03220772],
       [ 0.0469465 ,  0.05389679, -0.08643897, ...,  0.07224295,
        -0.00073808,  0.03869221],
       [ 0.04877882,  0.05384318, -0.10464869, ...,  0.10165274,
        -0.07252502,  0.0408757 ]], dtype=float32)

In [52]:
print(embed_weights.shape) #same size as vocab size and embedding dim (output dim of our embedding layer)

(10000, 128)


Now we have got the embedding matrix our model has learned to represent our tokens, lets see how we can visualize it. 

To do so, TensorFlow has a handy tool called projector :http://projector.tensorflow.org/

And tensorflow also has a incredible guide on word embeddings themselves: https://www.tensorflow.org/text/guide/word_embeddings

In [53]:
# #create embedding files (we got this from tensorflwo word embeddings documentation)
# import io
# out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
# out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

# for index, word in enumerate(words_in_vocab):
#   if index == 0:
#     continue  # skip 0, it's padding.
#   vec = embed_weights[index]
#   out_v.write('\t'.join([str(x) for x in vec]) + "\n")
#   out_m.write(word + "\n")
# out_v.close()
# out_m.close()

In [54]:
# #download files from colab to projector (http://projector.tensorflow.org/)
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

Downloading the files above we can visualize them using http://projector.tensorflow.org/ and clicking the "load" button on the left hand same

>Resouces: If you would like to know more about embeddings. I would encourage you to check out:
* https://jalammar.github.io/illustrated-word2vec/
* https://www.tensorflow.org/text/guide/word_embeddings

## Recurrent Neural Networks (RNN's)

RNN are useful for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

https://colah.github.io/posts/2015-08-Understanding-LSTMs/

## Model 2: LSTM

LSTM= long short term memory (one of the most popular LSTM cells)

Our structure of an RNN typically looks like this: 

```
Input(text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (label probabilities)
```

In [55]:
#create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#print(x.shape)
#x = layers.LSTM(64, return_sequences=True)(x) #when you are stacking RNN cells together, you need to set return_sequences
#print(x.shape)
x = layers.LSTM(64)(x)
#print(x.shape)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [56]:
# get a summary
model_2.summary()


Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                      

In [57]:
#compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [58]:
#fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220306-064825
Epoch 1/5
215/215 [==============================] - 10s 13ms/step - loss: 0.2218 - accuracy: 0.9241 - val_loss: 0.6485 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1552 - accuracy: 0.9437 - val_loss: 0.6082 - val_accuracy: 0.7900
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1293 - accuracy: 0.9504 - val_loss: 0.6727 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1065 - accuracy: 0.9588 - val_loss: 0.7422 - val_accuracy: 0.7822
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0848 - accuracy: 0.9663 - val_loss: 1.0052 - val_accuracy: 0.7822


In [59]:
#make prediction with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[7.6010145e-02],
       [9.0745366e-01],
       [9.9998426e-01],
       [7.5888038e-02],
       [1.5738462e-04],
       [9.9984813e-01],
       [9.8456037e-01],
       [9.9998772e-01],
       [9.9996531e-01],
       [8.1049222e-01]], dtype=float32)

In [60]:
#convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [61]:
#calculate model 2 results
model_2_results= calculate_results(y_true=val_labels,
                                   y_pred= model_2_preds)

In [62]:
model_2_results

{'accuracy': 78.21522309711287,
 'f1': 0.7815779758842515,
 'precision': 0.7818857870038972,
 'recall': 0.7821522309711286}

In [63]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 3: GRU

Another popular and effective RNN component is the GRU or Gated Recurrent Unit.

The GRU cell has similar features to an LSTM cell but has less parameters

In [64]:
#Build an RNN using the GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
#x = layers.GRU(64, return_sequences=True)(x) #put return_sequences=True if stacking the recurrent layers
# x = layers.LSTM(64,return_sequences=True)(x)
# x = layers.GRU(64)(x)
# x = layers.Dense(64,activation="relu")(x)

outputs = layers.Dense(1,activation = "sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [65]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [66]:
#compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [67]:
#fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data= (val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20220306-064842
Epoch 1/5
215/215 [==============================] - 3s 8ms/step - loss: 0.1540 - accuracy: 0.9397 - val_loss: 0.7078 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0808 - accuracy: 0.9702 - val_loss: 0.8626 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0701 - accuracy: 0.9717 - val_loss: 0.9149 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0616 - accuracy: 0.9733 - val_loss: 1.1759 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0528 - accuracy: 0.9778 - val_loss: 1.1283 - val_accuracy: 0.7717


In [68]:
#make prediction with our GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[1.11799024e-03],
       [8.76308143e-01],
       [9.99923944e-01],
       [1.67850390e-01],
       [1.05321946e-04],
       [9.99772966e-01],
       [9.59098935e-01],
       [9.99955058e-01],
       [9.99927640e-01],
       [9.76863682e-01]], dtype=float32)

In [69]:
#convert model 3 pred probs to labels
model_3_preds= tf.squeeze(tf.round(model_3_pred_probs))

In [70]:
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [71]:
#calculate model 3 result
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred =model_3_preds)

model_3_results

{'accuracy': 77.16535433070865,
 'f1': 0.7702930138828643,
 'precision': 0.7720646185150423,
 'recall': 0.7716535433070866}

## Model 4: Bidirectional RNN

Normal RNN's go from left to right (just like you would read an english snetences), however bidirectional RNN goes from right to left as well as left to right.

Refer: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional

In [78]:
#Build a biirectional RNN in tensorflow
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
# x = layers.Bidirectional(layers.GRU(64))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [79]:
#get summary 
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_6 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [80]:
#compile the model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [83]:
#fit the model
model_4_history=model_4.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences, val_labels),
                            callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                   "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20220306-070858
Epoch 1/5
215/215 [==============================] - 8s 11ms/step - loss: 0.1075 - accuracy: 0.9699 - val_loss: 0.9636 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0521 - accuracy: 0.9759 - val_loss: 1.0020 - val_accuracy: 0.7572
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0478 - accuracy: 0.9793 - val_loss: 1.2332 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0410 - accuracy: 0.9800 - val_loss: 1.3964 - val_accuracy: 0.7717
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0444 - accuracy: 0.9804 - val_loss: 1.4379 - val_accuracy: 0.7690


In [84]:
#make prediction
model_4_pred_probs=model_4.predict(val_sentences)

In [85]:
model_4_pred_probs[:10]

array([[8.1609167e-02],
       [9.5719522e-01],
       [9.9998844e-01],
       [1.8421124e-01],
       [4.3032142e-05],
       [9.9991953e-01],
       [9.8203665e-01],
       [9.9998975e-01],
       [9.9998438e-01],
       [9.9721766e-01]], dtype=float32)

In [86]:
#convert pred probs to pred labels
model_4_preds= tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [88]:
#calcualte the results of our bidirectional model
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_preds)

In [89]:
model_4_results

{'accuracy': 76.9028871391076,
 'f1': 0.768063295978387,
 'precision': 0.7689137548517897,
 'recall': 0.7690288713910761}

## Convolutional Neural Network for text (and other types of sequences)

we have used CNNs for images are typically 2D (height x width).. however, our text data is 1D.

Conv2D we ahve Conv2D for our image data but now we are gooing to use Conv1D.

The typical structure of a Conv1D model for sequences (in our case, text):
```
Inputs (text)-> Tokenization -> Embedding -> Layer(s) -> (typically Conv1D +pooling) -> Outputs (class probabilities)
```

### Model 5: Conv1D

refer: https://poloclub.github.io/cnn-explainer/

In [97]:
#Test out our embedding layer, Conv1D layer and max pooling
from tensorflow.keras import layers
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) #turn target sentence into embedding
conv_1d= layers.Conv1D(filters=32,
                       kernel_size=5, #this is also referred to as an ngram of 5
                       strides=1, #default
                       activation="relu",
                       padding="valid") #default="valid", the output is smaller than input

conv_1d_output = conv_1d(embedding_test) #pas test embedding through conv1d layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output=max_pool(conv_1d_output) #equivalent to "get the most importnat feature" or get the highest value

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [98]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02131199, -0.01554646, -0.0782732 , ..., -0.00444955,
         -0.0705274 ,  0.06940973],
        [ 0.03634493, -0.00982067, -0.03024419, ..., -0.03411037,
          0.03802231,  0.01045206],
        [-0.01998089,  0.04638628,  0.00227828, ..., -0.01783872,
         -0.03907752, -0.01996588],
        ...,
        [-0.00594839, -0.02332556, -0.00316688, ..., -0.0022279 ,
          0.01698233, -0.01680881],
        [-0.00594839, -0.02332556, -0.00316688, ..., -0.0022279 ,
          0.01698233, -0.01680881],
        [-0.00594839, -0.02332556, -0.00316688, ..., -0.0022279 ,
          0.01698233, -0.01680881]]], dtype=float32)>

In [99]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 7.73528591e-03, 0.00000000e+00, 0.00000000e+00,
         5.90206757e-02, 5.59743643e-02, 1.32547505e-02, 7.99385551e-03,
         8.99607390e-02, 5.41172475e-02, 1.33957565e-02, 1.82809941e-02,
         4.24990207e-02, 4.61281929e-03, 0.00000000e+00, 0.00000000e+00,
         3.58146653e-02, 1.94614716e-02, 3.97631973e-02, 0.00000000e+00,
         1.89017542e-02, 3.43388468e-02, 0.00000000e+00, 1.46786636e-02,
         2.63114721e-02, 0.00000000e+00, 0.00000000e+00, 2.45608483e-02],
        [4.80075255e-02, 9.28441621e-03, 2.90133189e-02, 0.00000000e+00,
         3.41011211e-02, 0.00000000e+00, 0.00000000e+00, 5.01515381e-02,
         3.25778313e-02, 0.00000000e+00, 5.69756366e-02, 0.00000000e+00,
         8.08307715e-03, 0.00000000e+00, 4.13264800e-03, 0.00000000e+00,
         4.62510958e-02, 0.00000000e+00, 0.00000000e+00, 1.42075568e-0

In [100]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.04800753, 0.00928442, 0.02901332, 0.        , 0.09025913,
        0.03104575, 0.0226765 , 0.06071973, 0.05902068, 0.05597436,
        0.06124651, 0.07956116, 0.08996074, 0.07374068, 0.04071477,
        0.06192271, 0.0462511 , 0.02674996, 0.01205009, 0.03551088,
        0.06497794, 0.05192376, 0.0397632 , 0.0194474 , 0.01890175,
        0.03433885, 0.03874993, 0.04835185, 0.09163325, 0.04958799,
        0.        , 0.02456085]], dtype=float32)>